In [3]:
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import spacy

AttributeError: module 'pyarrow' has no attribute '__version__'

In [ ]:
nlp = spacy.load('pt_core_news_sm')

def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text

In [ ]:
df = pd.read_csv('pre-processed.csv',sep=';')
df = df[['preprocessed_news','label']]
df = df.dropna()
df['label'] = df['label'].astype(int)
df['preprocessed_news'] = df['preprocessed_news'].apply(lemmatize_text)
base = df.rename(columns={'preprocessed_news': 'Noticia'})
base

In [ ]:


# Suponha que temos um conjunto de dados de revisões de filmes
reviews = [...]  # Lista de revisões de filmes
labels = [...]  # Lista de rótulos ('positivo' ou 'negativo')

# Pré-processamento de texto e treinamento do modelo Word2Vec
sentences = [review.split() for review in reviews]
word2vec = Word2Vec(sentences, min_count=1)

# Preparando os dados de entrada para a rede neural
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews)
sequences = tokenizer.texts_to_sequences(reviews)
data = pad_sequences(sequences)

# Preparando a matriz de incorporação
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, word2vec.vector_size))
for word, i in tokenizer.word_index.items():
    embedding_matrix[i] = word2vec[word]

# Construindo a rede neural
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, word2vec.vector_size, weights=[embedding_matrix]))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Dividindo os dados em treinamento e teste
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

# Treinando a rede neural
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10)
